In [1]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split

/home/pranavk/Documents/github/kitts-llm/venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-05 13:55:32.849874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736114132.861105  207488 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736114132.864614  207488 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 13:55:32.878167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in p

In [2]:
window_size = 128
stride = 1
embedding_dim = 512
num_layers = 24
num_heads = 16
ff_dim = 1024
train_ratio = 0.8

In [3]:
model_name = "EleutherAI/gpt-neo-125M"
model = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [4]:
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def load_essays_from_folder(folder_path):
    essays = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as f:
                essays.append(f.read())
    return essays 

essays = load_essays_from_folder('rawdata/essays')
essays[:5]

['I remember the first time I grasped the power of natural language processing (NLP). In 9th grade, I was toying around with Google Translate when I began wondering how it works. To an end user, it might look simple—type in a sentence, hit enter, and voilà, the machine gives you a translation for any of its 234 languages. I couldn\'t help but think, How is this even possible? Language is so messy and full of nuances.\nThis fascination resonated with my love for math—I find myself searching for numerical patterns, whether factoring street numbers or playing "24" with words (where A=1, B=2, and so on). Through NLP, I discovered how computers could transform language into mathematical representations, combining my love of patterns with real-world impact.\nMy passion grew through hands-on projects. After completing Stanford\'s NLP course on Coursera, I sought mentorship from Krishna Chintalapudi, a Principal Researcher at Microsoft, to improve text classification techniques. For Technology

In [6]:
def generate_sliding_window_data(sequence):
    inputs = []
    labels = []

    for i in range(0, len(sequence) - window_size, stride):
        window_sequence = sequence[i:i + window_size]
        next_tokens = sequence[i + 1:i + window_size + 1]  # Shift by one for labels
        inputs.append(window_sequence)
        labels.append(next_tokens)

    return inputs, labels

In [7]:
def prepare_data(essays):
    inputs = []
    labels = []

    for essay in essays:
        tokenized_essay = tokenizer(
                                    essay,
                                    padding="max_length",
                                    return_tensors="np",
                                )["input_ids"][0]
        essay_inputs, essay_labels = generate_sliding_window_data(tokenized_essay)
        inputs.extend(essay_inputs)
        labels.extend(essay_labels)

    return np.array(inputs), np.array(labels)

In [8]:
inputs, labels= prepare_data(essays)

In [9]:
inputs.shape, labels.shape

((94080, 128), (94080, 128))

In [10]:
class KittuSLMData(Dataset):
    def __init__(self, tokenized_inputs, tokenized_labels):
        self.input_ids = tokenized_inputs
        self.labels = tokenized_labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx], dtype=torch.long),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# Create dataset
dataset = KittuSLMData(inputs, labels)

In [11]:
train_Size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_Size

In [12]:
train_dataset, val_dataset = random_split(dataset, [train_Size, val_size])
len(train_dataset), len(val_dataset)

(75264, 18816)

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True
)

/home/pranavk/Documents/github/kitts-llm/venv/lib64/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [15]:
trainer.train()

  2%|▏         | 100/4704 [00:33<25:41,  2.99it/s]

{'loss': 0.9669, 'grad_norm': 1.2045801877975464, 'learning_rate': 4.8937074829931974e-05, 'epoch': 0.02}


  4%|▍         | 200/4704 [01:07<25:42,  2.92it/s]

{'loss': 0.5589, 'grad_norm': 1.6572766304016113, 'learning_rate': 4.7874149659863945e-05, 'epoch': 0.04}


  6%|▋         | 300/4704 [01:41<25:58,  2.83it/s]

...

                                                   
100%|██████████| 4704/4704 [28:20<00:00,  3.04it/s]

{'eval_loss': 0.023786062374711037, 'eval_runtime': 134.2979, 'eval_samples_per_second': 140.106, 'eval_steps_per_second': 8.757, 'epoch': 1.0}


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
100%|██████████| 4704/4704 [28:30<00:00,  2.75it/s]

{'train_runtime': 1710.7934, 'train_samples_per_second': 43.994, 'train_steps_per_second': 2.75, 'train_loss': 0.0894479629057808, 'epoch': 1.0}


TrainOutput(global_step=4704, training_loss=0.0894479629057808, metrics={'train_runtime': 1710.7934, 'train_samples_per_second': 43.994, 'train_steps_per_second': 2.75, 'total_flos': 4914872743624704.0, 'train_loss': 0.0894479629057808, 'epoch': 1.0})

In [18]:
prompt = "I remember the first time I grasped the power of natural language processing (NLP). In 9th grade, I was toying around with Google Translate when I began wondering how it works."
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

output = model.generate(
    input_ids=input_ids,
    max_length=256,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.9
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I remember the first time I grasped the power of natural language processing (NLP). In 9th grade, I was toying around with Google Translate when I began wondering how it works. an an, anof,of isofd that, an conducted, turned, took place Google a,,,,a,,,,,,,,,,,,,,,,,,,,,,,, Googlelate,., an a where, was it an about whether to or not.in an inth, inth, at, place I to, that my showed, me the, that I the, that me the I create
